In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [155]:
classes = pd.read_csv('open_classes.csv')
classes.head()

,Unnamed: 0,년도,학기,학수번호,분반,이수구분,개설학과,교과목명,담당교수,학점(시간),강의시간/강의실,상대평가,인원제한,교환학생,출석확인자율화,무감독시험,유연학기,학부/대학원구분
0,0,2019,1R,ACEE121,00,전공필수,건축사회환경공학부,건축사회환경공학의미래,최항석,3(3),월(5) 창의관 B113호 수(5) 창의관 B113호,N,Y,Y,N,N,N,학부
1,1,2019,1R,ACEE123,00,전공선택,건축사회환경공학부,건축시스템의이해,강경인,3(3),월(6-8) 창의관 207호,Y,Y,N,N,N,N,학부
2,2,2019,1R,ACEE219,00,전공선택,건축사회환경공학부,환경화학(영강),안창훈,3(3),화(1) 신공학관B104호 목(1) 신공학관B104호,N,Y,N,Y,N,N,학부
3,3,2019,1R,ACEE221,01,전공필수,건축사회환경공학부,공업수학I(영강),이종섭,3(3),월(1) 과도관 615호 수(1) 과도관 615호,N,Y,N,N,N,N,학부
4,4,2019,1R,ACEE221,02,전공필수,건축사회환경공학부,공업수학I(영강),이재상,3(3),월(1) 과도관 410호 수(1) 과도관 410호,N,Y,N,N,N,N,학부


In [156]:
classes.columns

Index(['Unnamed: 0', '년도', '학기', '학수번호', '분반', '이수구분', '개설학과', '교과목명', '담당교수',
       '학점(시간)', '강의시간/강의실', '상대평가', '인원제한', '교환학생', '출석확인자율화', '무감독시험',
       '유연학기', '학부/대학원구분'],
      dtype='object')

In [157]:
classes.drop(['Unnamed: 0'], axis=1, inplace=True)

In [158]:
#문장 단위별로 split해서 빈 리스트인 temp에 append
temp = []
for sent in classes['강의시간/강의실']:
    temp.append(sent.split())

temp

[['월(5)', '창의관', 'B113호', '수(5)', '창의관', 'B113호'],
 ['월(6-8)', '창의관', '207호'],
 ['화(1)', '신공학관B104호', '목(1)', '신공학관B104호'],
 ['월(1)', '과도관', '615호', '수(1)', '과도관', '615호'],
 ['월(1)', '과도관', '410호', '수(1)', '과도관', '410호'],
 ['월(1)', '공학관', '167호', '수(1)', '공학관', '167호'],
 ['월(2)', '신공학관B101', '화(9)', '신공학관B101', '수(2)', '신공학관B101'],
 ['월(2)', '과도관', '410호', '화(9)', '과도관', '410호', '수(2)', '과도관', '410호'],
 ['화(5)', '신공학관B101', '목(4-5)', '신공학관B101'],
 ['화(2-3)', '아산이학관', '331호', '목(2-3)', '아산이학관', '331호'],
 ['월(5)', '창의관', '207호', '수(5)', '창의관', '207호'],
 ['목(6-7)', '공학관', '167호', '토(2)', '공학관', '167호'],
 ['금(4-6)', '공학관', '250호'],
 ['월(1)', '신공학관B105', '월(4)', '신공학관B105', '수(1)', '신공학관B105'],
 ['화(2)', '창의관', '116호', '목(2)', '창의관', '116호'],
 ['월(7-8)', '신공학관B104호', '수(7-8)', '신공학관B104호'],
 ['목(7-10)', '창의관', '127호'],
 ['금(1-3)', '창의관', '127호'],
 ['월(2-3)',
  '신공학관B109-전산실습실',
  '월(2-3)',
  '창의관',
  '127호',
  '수(2-3)',
  '신공학관B109-전산실습실',
  '수(2-3)',
  '창의관',
  '127호'],
 ['월(6)', '과도관', '3

<b>데이터셋 구성 (경영, 전전, 핵심교양별로 데이터셋 다르게 구성): </b><br>
학수번호|분반|교과목명|담당교수|강의시간/강의실|건물(창의관)|요일(월(1)수(2))|층(1)
- len=1: ['수(9-11)'] (교시만 나와있는 경우)
- len=3: ['월(6-8)', '창의관', '207호'] (1개의 교시만 있는 경우)
- len=4: ['화(5)', '신공학관B105', '목(5)', '신공학관B105'] (지하인 경우, B로 표기))
- len=6: ['월(1)', '과도관', '615호', '수(1)', '과도관', '615호']
- len=8: ['월(2)', '하나과학관', '지하', '127호', '수(2)', '하나과학관', '지하', '127호'] (지하인 경우, 지하로 표기)
- len=9: ['화(2-3)', '창의관', '101호', '화(2-3)', '창의관', '127호', '목(2)', '창의관', '127호'] 

In [159]:
#1) 요일 추출
for i in range(len(temp)):
    if len(temp[i]) == 1:
        classes.loc[i, '요일'] = ''.join(temp[i][0]) #구분자 띄어쓰기로 문자열 합쳐주기
    elif len(temp[i]) == 3:
        classes.loc[i, '요일'] = ''.join(temp[i][0])
    elif len(temp[i]) == 4:
        classes.loc[i, '요일'] = ''.join([temp[i][0], temp[i][2]]) #1번째, 3번째 열
    elif len(temp[i]) == 6:
        classes.loc[i, '요일'] = ''.join([temp[i][0], temp[i][3]]) #1번째, 4번째 열
    elif len(temp[i]) == 8:
        classes.loc[i, '요일'] = ''.join([temp[i][0], temp[i][4]])
    elif len(temp[i]) == 9:
        classes.loc[i, '요일'] = ''.join([temp[i][0], temp[i][3], temp[i][6]])
    else:
        classes.loc[i, '요일'] = ''.join(temp[i]) #있는 그대로 출력

#### [정규표현식 종류]
- 별표(*): 앞 문자와 0회 이상 일치 (모두 불러오기)
- 더하기(+): 앞 문자와 1회 이상 일치 (1개 이상인 경우 불러오기)
- 점(.): 아무거나 와도됨 (.호인 경우: 호 이전에 아무거나 와도 됨)

In [160]:
#뒤에 '지하'라고 붙어있는 경우 없애기 위해 정규표현식 사용, NULL은 N대신 공백처리

for i in range(len(temp)):
    time = classes['요일'][i] 
    if (re.findall(r'\w\(\d-\d*\)', time) != []): 
    #word(알파벳/숫자) + (숫자-숫자)의 형태, 별표(*)는 앞 문자와 0회 이상 일치함을 의미
    #null이 아닌 경우만 추출
        classes.loc[i, '요일'] = ''.join(re.findall(r'\w\(\d-\d*\)', time)) #\w(\d-\d*\) = 월(6-8)
    elif (re.findall(r'\w\(\d\)', time) != []):
        classes.loc[i, '요일']= ''.join(re.findall(r'\w\(\d\)', time)) #\w\(\d\) = 월(5), 수(6)
    else:
        classes.loc[i, '요일'] = 'NULL'

In [161]:
#2) 층 추출
for i in range(len(temp)):
    floor = classes['강의시간/강의실'][i]
    if (re.findall(r'\d+.호', floor) != []): #null이 아니고 (숫자)호 형태인 경우, 더하기(+)는 앞 문자와 1회 이상 일치함을 의미
        classes.loc[i, '층'] = ''.join(re.findall(r'\d+.호', floor)[0][0]) #1번째 문자만 가져오기
    elif (re.findall(r'\d+.호', floor) == []) & (re.findall(r'\w\d+', floor) != []): #신공학관B102/b102인 경우
        classes.loc[i, '층'] = ''.join(re.findall(r'\w\d+', floor)[0][1]) #2번째 문자만 가져오기
    elif (re.findall(r'B\d+', floor) != []): #B102인 경우 (대문자)
        classes.loc[i, '층'] = ''.join(re.findall(r'B\d+', floor)[0][0:2])
    else:
        classes.loc[i, '층'] = 'NULL'

In [162]:
#3) 건물 추출
#3-1) 건물명 뒤에 '관'이 안 붙는 경우
#3-2) 건물명 뒤에 '관'이 붙는 경우

for i in range(len(temp)):
    place = classes['강의시간/강의실'][i]
    if (re.findall(r'\w+관', place) != []):
        classes.loc[i, '건물'] = ''.join(re.findall(r'\w+관', place)[0])
    elif (re.findall('L-P', place) != []):
        classes.loc[i, '건물'] = 'L-P'
    elif (re.findall('미래융합', place) != []):
        classes.loc[i, '건물'] = '미래융합'
    else:
        classes.loc[i, '건물'] = 'NULL'

In [163]:
classes = classes[['년도', '학기', '학수번호', '분반', '개설학과', '교과목명', '담당교수', '건물', '층', '요일']]
classes.head()

,년도,학기,학수번호,분반,개설학과,교과목명,담당교수,건물,층,요일
0,2019,1R,ACEE121,00,건축사회환경공학부,건축사회환경공학의미래,최항석,창의관,1,월(5)수(5)
1,2019,1R,ACEE123,00,건축사회환경공학부,건축시스템의이해,강경인,창의관,2,월(6-8)
2,2019,1R,ACEE219,00,건축사회환경공학부,환경화학(영강),안창훈,신공학관,1,화(1)목(1)
3,2019,1R,ACEE221,01,건축사회환경공학부,공업수학I(영강),이종섭,과도관,6,월(1)수(1)
4,2019,1R,ACEE221,02,건축사회환경공학부,공업수학I(영강),이재상,과도관,4,월(1)수(1)


In [164]:
classes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19299 entries, 0 to 19298
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   년도      19299 non-null  int64 
 1   학기      19299 non-null  object
 2   학수번호    19299 non-null  object
 3   분반      19295 non-null  object
 4   개설학과    19299 non-null  object
 5   교과목명    19299 non-null  object
 6   담당교수    19299 non-null  object
 7   건물      19299 non-null  object
 8   층       19299 non-null  object
 9   요일      19299 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.5+ MB


In [165]:
#2020-2학기 과목 추출
classes_20 = classes['년도'] == 2020
classes_20_2 = classes['학기'] == '2R'
classes_20_2r = classes[classes_20 & classes_20_2]
classes_20_2r

,년도,학기,학수번호,분반,개설학과,교과목명,담당교수,건물,층,요일
9830,2020,2R,ACEE224,00,건축사회환경공학부,토질역학,최항석,공학관,3,월(1)수(1)
9831,2020,2R,ACEE226,01,건축사회환경공학부,환경공학I(영강),김지형,창의관,1,월(2)수(2)
9832,2020,2R,ACEE226,02,건축사회환경공학부,환경공학I(영강),홍승관,신공학관,1,월(2)수(2)
9833,2020,2R,ACEE228,00,건축사회환경공학부,구조역학I(영강),주영규,과도관,4,수(4-5)
9834,2020,2R,ACEE230,00,건축사회환경공학부,하천공학및실험(영강),백경록,과도관,6,화(5)화(8)목(5)
...,...,...,...,...,...,...,...,...,...,...
13031,2020,2R,TEEN303,00,기술창업융합전공,혁신성장창업트렌드,한관수,과도관,4,목(5-6)
13032,2020,2R,TEEN306,00,기술창업융합전공,기술창업전략,정경희,하나과학관,2,화(5-6)
13033,2020,2R,TEEN312,01,기술창업융합전공,창업현장실습Ⅱ,주영규,NULL,NULL,NULL
13034,2020,2R,TEEN312,02,기술창업융합전공,창업현장실습Ⅱ,주영규,NULL,NULL,NULL


In [166]:
#경영학과와 전기전자공학부가 개설학과인 데이터 추출
#경영학과
business_class = classes_20_2r[classes_20_2r['개설학과'] == '경영학과']
business_class.head()

,년도,학기,학수번호,분반,개설학과,교과목명,담당교수,건물,층,요일
10018,2020,2R,BUSS152,01,경영학과,회계학원리(영강),김재영,L-P,4,월(1)수(1)
10019,2020,2R,BUSS152,02,경영학과,회계학원리(영강),권수영,L-P,4,월(5)수(5)
10020,2020,2R,BUSS152,03,경영학과,회계학원리,정석우,현차관,3,화(2)목(2)
10021,2020,2R,BUSS152,04,경영학과,회계학원리(영강),유용근,현차관,3,월(5)수(5)
10022,2020,2R,BUSS152,05,경영학과,회계학원리(영강),유용근,현차관,3,월(2)수(2)


In [167]:
#전기전자공학부
electronic_class = classes_20_2r[classes_20_2r['개설학과'] == '전기전자공학부']
electronic_class.head()

,년도,학기,학수번호,분반,개설학과,교과목명,담당교수,건물,층,요일
11949,2020,2R,KECE204,01,전기전자공학부,전기회로Ⅱ,엄두섭,공학관,5,월(5)수(5)
11950,2020,2R,KECE204,02,전기전자공학부,전기회로Ⅱ(영강),이상현,아산이학관,2,월(5)수(5)
11951,2020,2R,KECE204,03,전기전자공학부,전기회로Ⅱ(영강),최승연,아산이학관,1,월(5)수(5)
11952,2020,2R,KECE206,01,전기전자공학부,전자기학,김문일,공학관,1,월(1)수(1)
11953,2020,2R,KECE206,02,전기전자공학부,전자기학,전상근,창의관,1,월(2)수(2)


#### [핵심교양] 학수번호 패턴 탐지
- 과학과 기술: GEST
- 문학과 예술: GELA
- 사회의 이해: GESO
- 세계의 문화: GEFC
- 역사의 탐구: GEHI
- 윤리와 사상: GECE
- 정량적 사고: GEQR

In [184]:
#핵심교양
science = classes_20_2r['학수번호'].str.contains('GEST')
art = classes_20_2r['학수번호'].str.contains('GELA')
social = classes_20_2r['학수번호'].str.contains('GESO')
world = classes_20_2r['학수번호'].str.contains('GEFC')
history = classes_20_2r['학수번호'].str.contains('GEHI')
custom = classes_20_2r['학수번호'].str.contains('GECE')
think = classes_20_2r['학수번호'].str.contains('GEQR')
electives = classes_20_2r[science | art | social | world | history | custom | think]
electives.head()

,년도,학기,학수번호,분반,개설학과,교과목명,담당교수,건물,층,요일
10569,2020,2R,GECE107,00,교양교육원,진리·정의·자유를향한인문학적성찰,이진한,NULL,NULL,목(6-8)
10571,2020,2R,GEFC012,00,교양교육원,영화와언어,강범모,교육관,2,월(3-4)수(3-4)
10572,2020,2R,GEFC054,00,교양교육원,동남아시아의지역성과문화,서태열,교육관,2,월(6)수(6)
10573,2020,2R,GEFC076,00,교양교육원,유라시아한인문화탐방,김진규,교양관,2,화(5)목(5)
10574,2020,2R,GEFC091,00,교양교육원,종교와영화,박종천,교양관,6,월(2)수(2)


In [188]:
#경영, 전전, 핵교 모두 csv로 내보내기
business_class.to_csv('business_classes.csv', header=True, encoding='utf-8')
electronic_class.to_csv('electronic_classes.csv', header=True, encoding='utf-8')
electives.to_csv('electives.csv', header=True, encoding='utf-8')

In [11]:
#요일 부분은 요일1, 시작교시1, 끝교시1, 요일2, 시작교시2, 끝교시2 컬럼으로 나누기
import pandas as pd
import re

electronic = pd.read_csv("electronic_classes.csv")
business = pd.read_csv("business_classes.csv")
elective = pd.read_csv("electives.csv")

In [13]:
#1) 전기전자공학부

for i in range(len(electronic)): #교시(숫자만 추출)
    time = electronic['요일'][i]
    electronic.loc[i, '교시'] = ''.join(re.findall(r'\d+', str(time))) 

In [15]:
#교시(시작1, 끝1, 시작2, 끝2)
for i in range(len(electronic)):
    time = electronic['교시'][i]
    if len(time)==2: #11(화1교시, 목1교시)인 경우
        electronic.loc[i, '시작1']=''.join(time[0])
        electronic.loc[i, '끝1']=''.join(time[1])
        electronic.loc[i, '시작2']=''.join(time[0])
        electronic.loc[i, '끝2']=''.join(time[1])
    elif len(time)==4: #7879 (화2교시, 목2교시)인 경우
        electronic.loc[i, '시작1']=''.join(time[0])
        electronic.loc[i, '끝1']=''.join(time[1])
        electronic.loc[i, '시작2']=''.join(time[2])
        electronic.loc[i, '끝2']=''.join(time[3])
    else: #교시 없는 경우
        electronic.loc[i, '시작1']='NULL'
        electronic.loc[i, '끝1']='NULL'
        electronic.loc[i, '시작2']='NULL'
        electronic.loc[i, '끝2']='NULL'     

In [17]:
for i in range(len(electronic)): #요일(한글 정규표현식 사용)
    time = electronic['요일'][i]
    electronic.loc[i, '요일'] = ''.join(re.findall(r'[가-힣]+', str(time)))

In [18]:
#요일(요일1, 요일2)
for i in range(len(electronic)):
    time = electronic['요일'][i]
    if len(time)==1: #1개의 요일(수)
        electronic.loc[i, '요일1'] = ''.join(time)
        electronic.loc[i, '요일2'] = 'NULL'
    elif len(time)==2: #2개의 요일(화목)
        electronic.loc[i, '요일1'] = ''.join(time[0])
        electronic.loc[i, '요일2'] = ''.join(time[1])
    elif len(time)==4: #4개의 요일(화화목목으로 이상하게 로깅된 경우)
        electronic.loc[i, '요일1'] = ''.join(time[0])
        electronic.loc[i, '요일2'] = ''.join(time[2])
    else: #요일 없는 경우
        electronic.loc[i, '요일1'] = 'NULL'
        electronic.loc[i, '요일2'] = 'NULL'

In [22]:
for i in range(len(electronic)): #예외 case 처리
    if electronic.loc[i, '요일2'] == 'NULL':
        electronic.loc[i, '시작2']='NULL'
        electronic.loc[i, '끝2']='NULL'

In [28]:
del electronic['Unnamed: 0']
del electronic['요일']
del electronic['교시']

#컬럼 순서변경
electronic = electronic[['년도', '학기', '학수번호', '분반', '개설학과', '교과목명', '담당교수', '건물', '층', '요일1', '시작1', '끝1', '요일2', '시작2', '끝2']]
#컬럼명 변경
electronic.columns = ['year', 'semester', 'code', 'sep', 'department', 'class_name', 'professor_name', 'building', 'floor', 'day1', 'day2', 'start1', 'end1', 'start2', 'end2']

electronic.head(5)

,year,semester,code,sep,department,class_name,professor_name,building,floor,day1,day2,start1,end1,start2,end2
0,2020,2R,KECE204,1,전기전자공학부,전기회로Ⅱ,엄두섭,공학관,5,월,5,5,수,5,5
1,2020,2R,KECE204,2,전기전자공학부,전기회로Ⅱ(영강),이상현,아산이학관,2,월,5,5,수,5,5
2,2020,2R,KECE204,3,전기전자공학부,전기회로Ⅱ(영강),최승연,아산이학관,1,월,5,5,수,5,5
3,2020,2R,KECE206,1,전기전자공학부,전자기학,김문일,공학관,1,월,1,1,수,1,1
4,2020,2R,KECE206,2,전기전자공학부,전자기학,전상근,창의관,1,월,2,2,수,2,2


In [29]:
#2) 경영학과

for i in range(len(business)): #교시(숫자만 추출)
    time = business['요일'][i]
    business.loc[i, '교시'] = ''.join(re.findall(r'\d+', str(time))) 

In [30]:
#교시(시작1, 끝1, 시작2, 끝2)
for i in range(len(business)):
    time = business['교시'][i]
    if len(time)==2: #11(화1교시, 목1교시)인 경우
        business.loc[i, '시작1']=''.join(time[0])
        business.loc[i, '끝1']=''.join(time[1])
        business.loc[i, '시작2']=''.join(time[0])
        business.loc[i, '끝2']=''.join(time[1])
    elif len(time)==4: #7879 (화2교시, 목2교시)인 경우
        business.loc[i, '시작1']=''.join(time[0])
        business.loc[i, '끝1']=''.join(time[1])
        business.loc[i, '시작2']=''.join(time[2])
        business.loc[i, '끝2']=''.join(time[3])
    else: #교시 없는 경우
        business.loc[i, '시작1']='NULL'
        business.loc[i, '끝1']='NULL'
        business.loc[i, '시작2']='NULL'
        business.loc[i, '끝2']='NULL'     

In [31]:
for i in range(len(business)): #요일(한글 정규표현식 사용)
    time = business['요일'][i]
    business.loc[i, '요일'] = ''.join(re.findall(r'[가-힣]+', str(time)))

In [32]:
#요일(요일1, 요일2)
for i in range(len(business)):
    time = business['요일'][i]
    if len(time)==1: #1개의 요일(수)
        business.loc[i, '요일1'] = ''.join(time)
        business.loc[i, '요일2'] = 'NULL'
    elif len(time)==2: #2개의 요일(화목)
        business.loc[i, '요일1'] = ''.join(time[0])
        business.loc[i, '요일2'] = ''.join(time[1])
    elif len(time)==4: #4개의 요일(화화목목으로 이상하게 로깅된 경우)
        business.loc[i, '요일1'] = ''.join(time[0])
        business.loc[i, '요일2'] = ''.join(time[2])
    else: #요일 없는 경우
        business.loc[i, '요일1'] = 'NULL'
        business.loc[i, '요일2'] = 'NULL'

In [33]:
for i in range(len(business)): #예외 case 처리
    if business.loc[i, '요일2'] == 'NULL':
        business.loc[i, '시작2']='NULL'
        business.loc[i, '끝2']='NULL'

In [34]:
del business['Unnamed: 0']
del business['요일']
del business['교시']

#컬럼 순서변경
business = business[['년도', '학기', '학수번호', '분반', '개설학과', '교과목명', '담당교수', '건물', '층', '요일1', '시작1', '끝1', '요일2', '시작2', '끝2']]
#컬럼명 변경
business.columns = ['year', 'semester', 'code', 'sep', 'department', 'class_name', 'professor_name', 'building', 'floor', 'day1', 'day2', 'start1', 'end1', 'start2', 'end2']

business.head(5)

,year,semester,code,sep,department,class_name,professor_name,building,floor,day1,day2,start1,end1,start2,end2
0,2020,2R,BUSS152,1,경영학과,회계학원리(영강),김재영,L-P,4.0,월,1,1,수,1,1
1,2020,2R,BUSS152,2,경영학과,회계학원리(영강),권수영,L-P,4.0,월,5,5,수,5,5
2,2020,2R,BUSS152,3,경영학과,회계학원리,정석우,현차관,3.0,화,2,2,목,2,2
3,2020,2R,BUSS152,4,경영학과,회계학원리(영강),유용근,현차관,3.0,월,5,5,수,5,5
4,2020,2R,BUSS152,5,경영학과,회계학원리(영강),유용근,현차관,3.0,월,2,2,수,2,2


In [35]:
#3) 핵심교양

for i in range(len(elective)): #교시(숫자만 추출)
    time = elective['요일'][i]
    elective.loc[i, '교시'] = ''.join(re.findall(r'\d+', str(time))) 

In [37]:
#교시(시작1, 끝1, 시작2, 끝2)
for i in range(len(elective)):
    time = elective['교시'][i]
    if len(time)==2: #11(화1교시, 목1교시)인 경우
        elective.loc[i, '시작1']=''.join(time[0])
        elective.loc[i, '끝1']=''.join(time[1])
        elective.loc[i, '시작2']=''.join(time[0])
        elective.loc[i, '끝2']=''.join(time[1])
    elif len(time)==4: #7879 (화2교시, 목2교시)인 경우
        elective.loc[i, '시작1']=''.join(time[0])
        elective.loc[i, '끝1']=''.join(time[1])
        elective.loc[i, '시작2']=''.join(time[2])
        elective.loc[i, '끝2']=''.join(time[3])
    else: #교시 없는 경우
        elective.loc[i, '시작1']='NULL'
        elective.loc[i, '끝1']='NULL'
        elective.loc[i, '시작2']='NULL'
        elective.loc[i, '끝2']='NULL'     

In [38]:
for i in range(len(elective)): #요일(한글 정규표현식 사용)
    time = elective['요일'][i]
    elective.loc[i, '요일'] = ''.join(re.findall(r'[가-힣]+', str(time)))

In [39]:
#요일(요일1, 요일2)
for i in range(len(elective)):
    time = elective['요일'][i]
    if len(time)==1: #1개의 요일(수)
        elective.loc[i, '요일1'] = ''.join(time)
        elective.loc[i, '요일2'] = 'NULL'
    elif len(time)==2: #2개의 요일(화목)
        elective.loc[i, '요일1'] = ''.join(time[0])
        elective.loc[i, '요일2'] = ''.join(time[1])
    elif len(time)==4: #4개의 요일(화화목목으로 이상하게 로깅된 경우)
        elective.loc[i, '요일1'] = ''.join(time[0])
        elective.loc[i, '요일2'] = ''.join(time[2])
    else: #요일 없는 경우
        elective.loc[i, '요일1'] = 'NULL'
        elective.loc[i, '요일2'] = 'NULL'

In [40]:
for i in range(len(elective)): #예외 case 처리
    if elective.loc[i, '요일2'] == 'NULL':
        elective.loc[i, '시작2']='NULL'
        elective.loc[i, '끝2']='NULL'

In [41]:
del elective['Unnamed: 0']
del elective['요일']
del elective['교시']

#컬럼 순서변경
elective = elective[['년도', '학기', '학수번호', '분반', '개설학과', '교과목명', '담당교수', '건물', '층', '요일1', '시작1', '끝1', '요일2', '시작2', '끝2']]
#컬럼명 변경
elective.columns = ['year', 'semester', 'code', 'sep', 'department', 'class_name', 'professor_name', 'building', 'floor', 'day1', 'day2', 'start1', 'end1', 'start2', 'end2']

elective.head(5)

,year,semester,code,sep,department,class_name,professor_name,building,floor,day1,day2,start1,end1,start2,end2
0,2020,2R,GECE107,0,교양교육원,진리·정의·자유를향한인문학적성찰,이진한,NaN,NaN,목,6,8,NULL,NULL,NULL
1,2020,2R,GEFC012,0,교양교육원,영화와언어,강범모,교육관,2.0,월,3,4,수,3,4
2,2020,2R,GEFC054,0,교양교육원,동남아시아의지역성과문화,서태열,교육관,2.0,월,6,6,수,6,6
3,2020,2R,GEFC076,0,교양교육원,유라시아한인문화탐방,김진규,교양관,2.0,화,5,5,목,5,5
4,2020,2R,GEFC091,0,교양교육원,종교와영화,박종천,교양관,6.0,월,2,2,수,2,2


In [43]:
#경영, 전전, 핵교 모두 csv로 내보내기
business.to_csv('business_preprocessed.csv', header=True, encoding='utf-8')
electronic.to_csv('electronic_preprocessed.csv', header=True, encoding='utf-8')
elective.to_csv('electives_preprocessed.csv', header=True, encoding='utf-8')